# Titanic: Machine Learning from Disaster

In [155]:
# https://www.kaggle.com/c/titanic

In [156]:
import sklearn
import pandas as pd

In [157]:
# Загружаем данные из файлов
train = pd.read_csv('./homework/train.csv')
test = pd.read_csv('./homework/test.csv')

In [158]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Содержание:
1. Подготовка данных
2. Импорт библиотек моделей
3. Подбор параметров для Random Forest
4. Подбор параметров для Gradient Boosting
5. Подбор параметров для Decision Tree
6. Подбор параметров для SVM
7. Подбор параметров для Logistic Regression
8. Формирование матрицы предсказаний нескольких моделей
9. Объединение предсказаний by Logistic Regression
10. Формирование файла для отправки

## Предобработка данных


.
 


In [159]:
# Заполняем пропуски в данных медианными 
# значениями факторов на обучающей выборке
train_median = train.median()
train_imp = train.fillna(train_median)
test_imp = test.fillna(train_median)

In [160]:
# Бинаризуем категориальные признаки
CATEGORY_COL = ['Sex', 'Pclass', 'Embarked']
train_dummies = pd.get_dummies(train_imp, columns=CATEGORY_COL, drop_first=True)
test_dummies = pd.get_dummies(test_imp, columns=CATEGORY_COL, drop_first=True)

In [161]:
train_dummies.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_male,Pclass_2,Pclass_3,Embarked_Q,Embarked_S
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,1,0,1,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,0,0,0,0,0
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,0,1,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,0,0,0,0,1
4,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,1,0,1,0,1


In [162]:
# Удаляем лишние столбцы
DROP_COL = ['PassengerId', 'Name', 'Ticket', 'Cabin']
TARGET_COL = 'Survived'
X_train = train_dummies.drop(DROP_COL + [TARGET_COL], axis=1)
y_train = train_dummies[TARGET_COL]
X_test = test_dummies.drop(DROP_COL, axis=1)

In [163]:
X_train.head()

,Age,SibSp,Parch,Fare,Sex_male,Pclass_2,Pclass_3,Embarked_Q,Embarked_S
0,22.0,1,0,7.2500,1,0,1,0,1
1,38.0,1,0,71.2833,0,0,0,0,0
2,26.0,0,0,7.9250,0,0,1,0,1
3,35.0,1,0,53.1000,0,0,0,0,1
4,35.0,0,0,8.0500,1,0,1,0,1


## Предсказание моделей для стеккинга


.


In [164]:
import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer


from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression


def cross_val_predict_proba(estimator, X_train, y_train):
    kfold = KFold(n_splits=4, shuffle=True, random_state=None)
    return cross_val_predict(estimator, X_train, y_train, cv=kfold, method='predict_proba')

# TODO: подобрать гиперпараметры для ансамблей

kfold = KFold(n_splits=4, shuffle=True, random_state=None)

### Подбор параметров модели №1 для Random_forest

In [165]:
grid = {'n_estimators':[10,50,100],
       'max_features':[3,5,9],
       'max_depth':[2,6,10],
       'min_samples_leaf':[10,30,70,100],
        'warm_start':[0,1]
       }

gridSearchRandomForest = GridSearchCV(RandomForestClassifier(), grid, cv = kfold, scoring = 'accuracy')



In [166]:
param_finder_RFC = gridSearchRandomForest.fit(X_train, y_train)


In [167]:
param_finder_RFC.best_params_

{'max_depth': 10,
 'max_features': 3,
 'min_samples_leaf': 10,
 'n_estimators': 50,
 'warm_start': 0}

In [168]:
param_finder_RFC.cv_results_

{'mean_fit_time': array([ 0.02326661,  0.01348603,  0.07285047,  0.07578194,  0.13200665,
         0.14666975,  0.01998466,  0.00867754,  0.07117414,  0.07860249,
         0.16326696,  0.14182955,  0.01558161,  0.01206535,  0.07394183,
         0.07439798,  0.14010257,  0.14336246,  0.01562983,  0.01526129,
         0.06981927,  0.06912053,  0.1439454 ,  0.13973749,  0.01826966,
         0.00868845,  0.07711422,  0.07470196,  0.14514339,  0.13849467,
         0.01953113,  0.01562983,  0.07173198,  0.06276095,  0.13650841,
         0.14161628,  0.01562524,  0.0156256 ,  0.09163487,  0.07203299,
         0.14526123,  0.14978099,  0.01562995,  0.02006257,  0.06406635,
         0.06260234,  0.12648255,  0.13060087,  0.01562101,  0.01191688,
         0.08512491,  0.09656799,  0.2060762 ,  0.17655921,  0.01987559,
         0.02359712,  0.07812709,  0.07319051,  0.2104131 ,  0.19691771,
         0.03227127,  0.03577679,  0.08492017,  0.09465599,  0.23717558,
         0.18798423,  0.01224512, 

In [169]:
rf_estimator = RandomForestClassifier(max_depth=6,
                                  max_features= 3,
                                  min_samples_leaf= 10,
                                  n_estimators= 50,
                                  warm_start= 0)
rfc_estimator_fitted = rf_estimator.fit(X_train, y_train) 

##### Single Random Forest Classifier @ kaggle - 0.78947

### Подбор параметров модели №2 для GradientBoosting

In [170]:
grid = {'n_estimators':[3,10,50,100],
       'learning_rate':[0.03,0.3,3],
       'max_depth':[2,6,10],
       'min_samples_leaf':[10,30,70,100]
       }

gridSearchGradientBoosting = GridSearchCV(GradientBoostingClassifier(), grid, cv = kfold, scoring = 'accuracy')



In [171]:
param_finder_GBC = gridSearchGradientBoosting.fit(X_train, y_train)


C:\Users\Slava-N\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:256: RuntimeWarning: overflow encountered in multiply
  * tree.value[:, 0, 0].take(terminal_regions, axis=0))
C:\Users\Slava-N\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:490: RuntimeWarning: invalid value encountered in multiply
  np.sum(sample_weight * ((y * pred) - np.logaddexp(0.0, pred))))
C:\Users\Slava-N\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:256: RuntimeWarning: overflow encountered in multiply
  * tree.value[:, 0, 0].take(terminal_regions, axis=0))
C:\Users\Slava-N\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:490: RuntimeWarning: invalid value encountered in multiply
  np.sum(sample_weight * ((y * pred) - np.logaddexp(0.0, pred))))


In [172]:
param_finder_GBC.best_params_

{'learning_rate': 0.03,
 'max_depth': 10,
 'min_samples_leaf': 10,
 'n_estimators': 100}

In [173]:
gb_estimator = GradientBoostingClassifier(n_estimators=10, max_depth=6, min_samples_leaf=30, learning_rate=0.3)


In [174]:
gb_estimator_fitted = gb_estimator.fit(X_train, y_train)

##### Single GradientBoostingClassifier @ kaggle - 0.77512

### Подбор параметров модели №3 для Decision Tree

In [175]:
grid = {'max_depth':list(range(2,10,1)),
       'min_samples_leaf':[10,30,70,100]}

gridSearchDecisionTree = GridSearchCV(DecisionTreeClassifier(), grid, cv = kfold, scoring = 'accuracy')



In [176]:
param_finder_DT = gridSearchDecisionTree.fit(X_train, y_train)


In [177]:
param_finder_DT.best_params_

{'max_depth': 3, 'min_samples_leaf': 10}

In [178]:
dt_estimator = DecisionTreeClassifier(max_depth=3, min_samples_leaf=10)

In [179]:
dt_estimator_fitted = dt_estimator.fit(X_train, y_train)

##### Single Decision Tree @ kaggle - 0.78469

### Подбор параметров модели №4  для SVM

In [180]:
grid = {'C':[0.03, 0.3, 0.9, 3, 6, 9]}

gridSearchSupportVector = GridSearchCV(LinearSVC(), grid, cv = kfold, scoring = 'accuracy')

In [181]:
param_finder_SVC = gridSearchSupportVector.fit(X_train, y_train)


In [182]:
param_finder_SVC.best_params_

{'C': 0.3}

In [183]:
scores = cross_val_score(LinearSVC(C=0.03),
                         X_train, y_train, groups=None,
                        scoring = make_scorer(accuracy_score),
                        cv=kfold)
scores

array([ 0.80717489,  0.75784753,  0.84304933,  0.77927928])

In [184]:
svc_estimator = LinearSVC(C=0.03)

In [185]:
svc_estimator_fitted = svc_estimator.fit(X_train, y_train)

##### Single LinearSVC @ kaggle - 0.75
Самый плохой показатель, в стекинг моделей в любом случае не включаем - отсутствует метод оценки вероятности для предсказания.

### Подбор параметров модели №5  для LinearRegression

In [186]:
grid = {'C':[0.03, 0.3, 0.9, 3, 6, 9]}

gridSearchLogisticRegression = GridSearchCV(LogisticRegression(), grid, cv = kfold, scoring = 'accuracy')

In [187]:
param_finder_LR = gridSearchLogisticRegression.fit(X_train, y_train)

In [188]:
param_finder_LR.best_params_

{'C': 0.9}

In [189]:
LR_estimator = LogisticRegression(C=6)
LR_estimator_fitted = LR_estimator.fit(X_train, y_train)

##### Single LogisticRegression @ kaggle - 0.76077
Достаточно низкий показатель, в любом случае идет в стеккинг

## Формирование матрицы предсказаний четырех моделей

In [190]:
# получаем предсказания вероятностей ансамблей на кросс-валидации для обучающей выборки
rf_train_pred = cross_val_predict_proba(rf_estimator, X_train, y_train)
gb_train_pred = cross_val_predict_proba(gb_estimator, X_train, y_train)
dt_train_pred = cross_val_predict_proba(dt_estimator, X_train, y_train)
lr_train_pred = cross_val_predict_proba(LR_estimator, X_train, y_train)

In [203]:
X_train_stack = np.stack([rf_train_pred[:,1], gb_train_pred[:,1], dt_train_pred[:,1],lr_train_pred[:,1]], axis=1)

# получаем предсказания ансамблей для тестовой выборки
rf_test_pred = rfc_estimator_fitted.predict_proba(X_test)
gb_test_pred = gb_estimator_fitted.predict_proba(X_test)
dt_test_pred = dt_estimator_fitted.predict_proba(X_test)
lr_test_pred = LR_estimator_fitted.predict_proba(X_test)


X_test_stack = np.stack([rf_test_pred[:,1], gb_test_pred[:,1],dt_test_pred[:,1],lr_test_pred[:,1]], axis=1)


## Объединяем предсказания ансамблей с помощью логистической регрессии

In [204]:
grid = {'C':[0.03, 0.3, 0.9, 3, 6, 9,15,50]}
gridSearchEnsemble = GridSearchCV(LogisticRegression(), grid, cv = kfold, scoring = 'accuracy')

In [205]:
gridSearchEnsemble.fit(X_train_stack, y_train)

GridSearchCV(cv=KFold(n_splits=4, random_state=None, shuffle=True),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.03, 0.3, 0.9, 3, 6, 9, 15, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [206]:
gridSearchEnsemble.best_params_

{'C': 0.3}

При таком значении модель переобучается. Взято значение 0.5 - дает наилучший результат

50 0.7799
0.5 0.78794
0.03 0.78469

In [207]:
# TODO: подобрать гиперпараметры LogisticRegression

logreg = LogisticRegression(C=0.5).fit(X_train_stack, y_train)
predicted = logreg.predict(X_test_stack)

## Формируем фалй для отправки

In [202]:
with open('submission_ensemble_3_models_regularization_9.txt', 'w') as out:
    out.write('PassengerId,Survived\n')
    for passenger, y in zip(test['PassengerId'], predicted):
        out.write('%s,%s\n' % (passenger, y))